In [2]:
# подгружаем модуль re для работы с текстом
import re
import keras
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

In [3]:

# Загрузим текст
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый симво

In [4]:
# Теперь нам нужно разбить эти высказывания на слова. Для этого воспользуемся уже знакомым из прошлого занятия инструментом Tokenizer и 
# положим, что максимальное число слов будет равно 1000:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [5]:
# Итак, мы разбили текст на слова и для примера выведем их начальный список:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [6]:
# Далее, мы преобразуем текст в последовательность чисел в соответствии с полученным словарем. 
# Для этого используется специальный метод класса Tokenizer:
data = tokenizer.texts_to_sequences([texts])

In [7]:
# Осталось закодировать числа массива data в one-hot векторы. Для этого мы воспользуемся методом to_categorical пакета Keras:
res = to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


In [8]:
# Затем, из этой матрицы сформируем тензор обучающей выборки и соответствующий набор выходных значений. 
# Для начала вычислим размер обучающего множества:
inp_words = 3
n = res.shape[0]-inp_words

In [9]:
# И, далее, сформируем входной тензор и прогнозные значения также, как мы это делали с символами:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [10]:
# Все, у нас есть обучающая выборка и требуемые выходные значения. Осталось создать модель рекуррентной сети. 
# Мы ее возьмем из предыдущего занятия с числом нейронов скрытого слоя 128 и maxWordsCount нейронами на выходе с функцией активации softmax:
model_2 = Sequential()
model_2.add(Input((inp_words, maxWordsCount)))
model_2.add(SimpleRNN(128, activation='tanh'))
model_2.add(Dense(maxWordsCount, activation='softmax'))
model_2.summary()
 
model_2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               144512    
                                                                 
 dense (Dense)               (None, 1000)              129000    
                                                                 
Total params: 273,512
Trainable params: 273,512
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Настраиваем защиту от переобучения
erly_stoping_callback = EarlyStopping(monitor='accuracy', patience=3)  # Если метрика 'val_accuracy' продолжает снижение два шага подряд,
                                                                           # то выходим из процесса обучения

In [12]:

# Готово. Запускаем процесс обучения:
history_2 = model_2.fit(X, Y, batch_size=32, epochs=20, callbacks=[erly_stoping_callback])

Epoch 1/20
1/1 [==============================] - 1s 1s/step - loss: 6.9048 - accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 12ms/step - loss: 6.8787 - accuracy: 0.0357
Epoch 3/20
1/1 [==============================] - 0s 11ms/step - loss: 6.8523 - accuracy: 0.0714
Epoch 4/20
1/1 [==============================] - 0s 9ms/step - loss: 6.8253 - accuracy: 0.1786
Epoch 5/20
1/1 [==============================] - 0s 10ms/step - loss: 6.7975 - accuracy: 0.3571
Epoch 6/20
1/1 [==============================] - 0s 10ms/step - loss: 6.7685 - accuracy: 0.7143
Epoch 7/20
1/1 [==============================] - 0s 10ms/step - loss: 6.7382 - accuracy: 0.8929
Epoch 8/20
1/1 [==============================] - 0s 10ms/step - loss: 6.7062 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 9ms/step - loss: 6.6721 - accuracy: 1.0000
Epoch 10/20
1/1 [==============================] - 0s 9ms/step - loss: 6.6356 - accuracy: 1.0000
Epoch 11/20
1/1 [=============

In [13]:
# Смотрим на какой эпохе остановилось обучение
if erly_stoping_callback.stopped_epoch >=1:
    print('Обучение остановлено на эпохе', erly_stoping_callback.stopped_epoch)

Обучение остановлено на эпохе 10


In [14]:
# И давайте теперь посмотрим, что у нас получилось. Запишем функцию для формирования текста из спрогнозированных слов:
def buildPhrase(texts, str_len = 20):
  res = texts
  data = tokenizer.texts_to_sequences([texts])[0]

  for i in range(str_len):
    x = to_categorical(data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model_2.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
 
    res += " " + tokenizer.index_word[indx] # дописываем строку
 
  return res

In [15]:
# И вызовем ее с тремя первыми словами
res = buildPhrase('которые возникли в')

1/1 [==============================] - 0s 24ms/step


In [16]:
print(res)

которые возникли в понедельник думайте позитивно и верьте в свою способность достигать отличных результатов если вы смогли в понедельник подняться с постели значит
